## Index reference genome

In [ ]:
os.system('bwa index references.fasta')
os.system('samtools faidx references.fasta')
os.system('java -jar picard.jar CreateSequenceDictionary \
               R=references.fasta \
               O=references.dict')

## Alignment using speedseq

In [ ]:
os.system('speedseq align -o JT168_S62.filtered \
               -R "@RG\tID:id\tSM:JT168_S62\tLB:lib" \
               references.fasta \
               JT168_S62_R1_filtered.fastq.gz \
               JT168_S62_R2_filtered.fastq.gz')

In [ ]:
# samtools view -h -o JT168_S62.filtered.sam JT168_S62.filtered.bam

## Variant calling by GATK

In [ ]:
os.system('java -Xmx16g -jar GenomeAnalysisTK.jar -T HaplotypeCaller \
               -R references.fasta \
               -I JT168_S62.filtered.bam \
               -o JT168_S62.filtered.gatk.raw.vcf \
               -nct 8 \
               -allowPotentiallyMisencodedQuals \
               --allow_potentially_misencoded_quality_scores \
               --genotyping_mode DISCOVERY \
               -stand_call_conf 30 \
               -stand_emit_conf 10 \
               --defaultBaseQualities 30 \
               -variant_index_type LINEAR \
               -variant_index_parameter 128000')

## Read-based haplotype phasing with whatshap

In [ ]:
os.system('whatshap phase --indels --max-coverage 20 \
              -o JT168_S62.filtered.phased.vcf \
              JT168_S62.filtered.gatk.raw.vcf \
              JT168_S62.filtered.bam')

In [3]:
fh = open('JT168_S62.filtered.whatshap.phase.clean.txt', 'wt')


with open('JT168_S62.filtered.whatshap.phase.vcf') as f:
    for line in f:
        line = line.rstrip()
        
        if line.startswith('#'):
            continue
            
        lst = line.split('\t')
        
        fh.write('\t'.join([lst[0],lst[1],lst[3],lst[4], lst[-1]])+'\n')
        
fh.close()

## Read-based haplotype phasing with HapCut2

In [ ]:
### installation
git clone https://github.com/vibansal/HapCUT2.git
cd HapCUT2/
make

sudo make install-hairs
sudo make install-hapcut2

In [ ]:
### convert BAM file to the compact fragment file format containing only haplotype-relevant information. 
./build/extractHAIRS --bam ../JT168_S62.filtered.bam 
--VCF ../JT168_S62.filtered.gatk.raw.vcf 
--out ../JT168_S62.fragment_file

In [ ]:
### use HAPCUT2 to assemble fragment file into haplotype blocks.
./build/HAPCUT2 --fragments ../JT168_S62.fragment_file
--vcf ../JT168_S62.filtered.gatk.raw.vcf 
--output ../JT168_S62.filtered.hapcut2.phase.vcf

In [1]:
seq = {}


with open('JT168_S62.sam') as f:
    for line in f:
        line = line.rstrip()
        
        lst = line.split('\t')
        
        if lst[0].startswith('@'):
            continue
            
        else:
            if lst[2] not in seq:
                seq[lst[2]] = []
                
            seq[lst[2]].append(line)

In [3]:
for ke, val in seq.items():
    if ke == '*':
        continue
        
    else:
        fh = open(ke+'.sam', 'wt')
        
        fh.write('\n'.join(val))
        
        fh.close()
            